In [2]:
from kipoi_cadd.data_utils import load_csv_to_sparse_matrix
from kipoi_cadd.data import cadd_train_valid_data, train_test_split_indexes, CaddDataset, sparse_cadd_dataset, load_sparse_indexed_matrix
from kipoi_cadd.utils import dump_to_pickle, load_pickle
import pandas as pd
import time
from tqdm import tqdm, trange
from sklearn.model_selection import train_test_split
from kipoi_cadd.utils import get_dtypes_info
import numpy as np
from kipoi.data_utils import numpy_collate_concat
from scipy.sparse import csr_matrix, save_npz, load_npz
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
raw_training_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
pro_training_dir = "/s/project/kipoi-cadd/data/processed/v1.3/training_data/"
lmdb_dir = raw_training_dir + "lmdb"
csv_file = raw_training_dir + "training_data.imputed.csv"
csv_file_index = pro_training_dir + "training_data.imputed.index.csv"
valid_id_file = raw_training_dir + "valid_idx.pkl"
all_ids_file = raw_training_dir + "variant_ids.pkl"
ids_10k_file = raw_training_dir + "sample_indices/ids_10k.pkl"
std_scaler_file = raw_training_dir + "stats/standard_scaler_first10k.pkl"
short_csv = raw_training_dir + "shuffled_10k.csv"
short_csv_index = pro_training_dir + "shuffled_10k.index.csv"
sparse_all = raw_training_dir +  "sparse_all.npz"
sparse_short_index = raw_training_dir +  "sparse_short_indexed.npz"
sparse_short = raw_training_dir +  "sparse_short.npz"

## We need to make sure we know the order of the variants

In [64]:
def load_csv_to_sparse_matrix(csv_file, targets_col=0, blocksize=10E6, final_type=np.float32):
    import dask.dataframe as ddf
    from scipy.sparse import csr_matrix
    from dask.diagnostics import ProgressBar
    
    print("Started dask task.")
    df_dask = ddf.read_csv(csv_file, blocksize=blocksize, assume_missing=True)
    df_dask = df_dask.map_partitions(lambda part: part.to_sparse(fill_value=0))
    
    with ProgressBar():
        df_dask = df_dask.compute().reset_index(drop=True)
    
    print("Finished dask task.")
    csr = csr_matrix(df_dask, dtype=final_type, copy=True)
    print("Finished transforming to csr_matrix.")
    
    del df_dask
    
    print("Changing -1 in the targets...")
    y_array = csr[:, targets_row].toarray()
    y_array[y_array==-1] = 0
    new_y = csr_matrix(y_array)
    csr[:, targets_row] = new_y
    print("Done.")
    
    return csr

In [74]:
test = pd.read_csv(short_csv_index, index_col=0)

In [7]:
import os
sz = os.path.getsize(short_csv_index)
print(sz)
sz/100

46145182


461451.82

In [57]:
10E6

10000000.0

In [3]:
# csr = load_csv_to_sparse_matrix(short_csv_index, targets_col=1)
# save_npz(sparse_short_index, csr)
csr = load_npz(sparse_short_index)

In [75]:
trues = []
for i in range(100):
    trues.append(np.unique(csr[i,1:].toarray() == test.loc[i,:].values))
np.unique(trues)

array([ True])

### Now we test with the big file

In [36]:
csr_all = load_npz(sparse_all)
csv_all = pd.read_csv(csv_file_index, nrows=1000, index_col=0)

In [54]:
csr_all[i,1:].toarray().ravel().shape
csv_all.iloc[1,1:].values.shape

(1063,)

In [76]:
trues = []
for i in range(1000):
    trues.append(np.unique(csr_all[i,1:].toarray().ravel() == csv_all.iloc[i,1:].values))
np.unique(trues)

array([False,  True])

They're still not the same.

## Test retrieving the correct variant ids

In [4]:
new_csr, variant_ids = load_sparse_indexed_matrix(sparse_short_index, index_col=0, shuffle=False)
save_npz(sparse_short, new_csr)

Retrieved a matrix with shape  (10000, 1064)


In [14]:
# variant_ids
short_ids = load_pickle(raw_training_dir + "shuffle_splits/shuffled_index_10k.pkl")
all_ids = load_pickle(all_ids_file)
variant_names_short_ids = all_ids.loc[short_ids]
dump_to_pickle(raw_training_dir + "shuffle_splits/shuffled_index_10k.pkl", variant_names_short_ids)

## Now, use this with the CaddSparseDataset

In [5]:
from kipoi_cadd.data import CaddSparseDataset
sparse_ds = CaddSparseDataset(sparse_short, raw_training_dir + "shuffle_splits/shuffled_index_10k.pkl")

In [9]:
sparse_ds.__getitem__(10)
# sparse_ds.data[1,1]

{'inputs': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'targets': 0.0,
 'variant_id': "1:1253092:G:['C']"}